**4.1 DE**

Dense Matrix

Hamiltonian computation

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
import scipy
import scipy.sparse.linalg as sla
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.linalg import eigh
def ising_hamiltonian(L,h,periodic):
  H=np.zeros((2**L,2**L))
  for b in range(2**L):
    for j in range(1,L+1):
      a=b^(1<<j-1)
      H[a,b]=-h
  for a in range(2**L):
    for j in range(1,L):
      if a&(1<<j)==(a&(1<<j-1))*2:
        H[a,a]-=1
      else:
        H[a,a]+=1

    if periodic:
        if (a & (1 << L-1)) == ((a & (1 << 0))*(2**(L-1))) :
          H[a, a]-=1
        else:
          H[a, a]+=1
  return H

Comparison between periodic and non periodic

In [ ]:
L_values=[8,10,12]
def calculate_h_values(L):

    if L==8:
        return 50
    if L==10:
        return 30
    if L==12:
        return 30
    if L==14:
        return 12
    else:
        return 30
all_energies = {L: {'periodic': [], 'non_periodic': [], 'h_values': None} for L in L_values}

for periodic in [True, False]:
    for L in L_values:
        h_values = np.linspace(-2, 2, calculate_h_values(L))
        energies = []
        for h in tqdm(h_values, desc=f'Calculating for L={L}, Periodic={periodic}'):
            H = ising_hamiltonian(L, h, periodic)
            ground_state = scipy.linalg.eigh(H, subset_by_index=(0, 0), eigvals_only=True)
            energies.append(ground_state[0])
        all_energies[L]['h_values'] = h_values
        if periodic:
            all_energies[L]['periodic'] = energies
        else:
            all_energies[L]['non_periodic'] = energies

plt.figure(figsize=(14, 7))
for L, data in all_energies.items():
    plt.plot(data['h_values'], data['periodic'], label=f'L={L}, Periodic')
    plt.plot(data['h_values'], data['non_periodic'], linestyle='--', label=f'L={L}, Non-Periodic')

plt.xlabel('h values')
plt.ylabel('Ground state energies')
plt.title('Comparison of Ground State Energies')
plt.legend()
plt.show()


**4.2 Sparse matrix computation**

In [ ]:
from scipy.sparse import csr_matrix
def ising_sparse(L,h,periodic):
  rows = []
  cols = []
  matrix_elements = []
  for b in range(2**L):
    for j in range(1, L + 1):
      a = b ^ (1 << (j - 1))
      rows.append(a)
      cols.append(b)
      matrix_elements.append(-h)

  for a in range(2**L):
    diagonal_value =0
    for j in range(1, L):
        if a & (1 << j) == (a & (1 << (j - 1))) * 2:
            diagonal_value -= 1
        else:
            diagonal_value += 1

    if periodic:
        if (a & (1 << (L - 1))) == ((a & 1) * (2 ** (L - 1))):
            diagonal_value -= 1
        else:
            diagonal_value += 1
        rows.append(a)
        cols.append(a)
        matrix_elements.append(diagonal_value)
  H_sparse = csr_matrix((matrix_elements, (rows, cols)), shape=(2**L, 2**L), dtype=np.float64)
  return H_sparse


Plotting ground state energies for range of h

In [ ]:

import matplotlib.pyplot as plt
import scipy

L_values = [8,10,12,14,16,18,20]
h_values = [-1,0,1]
ground_state_energies = {L: [] for L in L_values}

for L in L_values:
    for h in tqdm(h_values, desc=f'Computing for L={L}'):
        H = ising_sparse(L, h, periodic=True)
        ground_state = scipy.sparse.linalg.eigsh(H, k=1, which='SA', return_eigenvectors=False)[0]
        ground_state_energies[L].append(ground_state)

for L, energies in ground_state_energies.items():
    plt.plot(h_values, energies, label=f'L={L}')
plt.xlabel('hvalues')
plt.ylabel('ground state energies')
plt.legend()
plt.show()

4.3 **Study of L dependence on ground state energy**

In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
from scipy.sparse.linalg import eigsh
from tqdm import tqdm
import scipy.sparse.linalg


h_values = [0.3, 1.7]
L_values = [4, 6, 8, 10, 12, 14, 16, 18, 20]

lists = []
for h in tqdm(h_valuess, desc='Processing h values'):

    ground_state_energy_open = []
    ground_state_energy_periodic = []

    for L in tqdm(L_values, desc=f'Processing L values for h={h}'):
        H_open = ising_sparse(L, h, 'open')
        H_periodic = ising_sparse(L, h, 'periodic')

        eigenvalues_open, _ = scipy.sparse.linalg.eigsh(H_open, which='SA')
        ground_state_energy_open.append(eigenvalues_open[0] / L)

        eigenvalues_periodic, _ = scipy.sparse.linalg.eigsh(H_periodic, which='SA')
        ground_state_energy_periodic.append(eigenvalues_periodic[0] / L)

    plt.title(f'h = {h}')
    plt.plot(L_values, ground_state_energy_open, label='open boundary')
    plt.plot(L_values, ground_state_energy_periodic, label='periodic boundary')
    plt.legend()
    plt.xlabel('L')
    plt.ylabel('Ground State Energy')
    plt.show()

    lists.append([(ground_state_energy_open[i+1]-ground_state_energy_open[i])/2 for i in range(len(ground_state_energy_open_)-1)])

for i in range(len(lists)):
    plt.plot(L_values[:-1], lists[i])
plt.title('Energy Differences')
plt.legend(['ferromagnetic', 'paramagnetic'])
plt.xlabel('L')
plt.ylabel('Ground State Energy')
plt.show()



**4.4 Identifying the quantum phase transition**

Plotting wavefunctions and computing nu

In [ ]:
import numpy as np
import scipy.sparse.linalg as sla
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
L_values = [4, 6, 8, 10, 12]
def find_excited_energies(H_sparse):
    eigenvalues = sla.eigsh(H_sparse, k=8, which='SA', return_eigenvectors=False)
    ground_state_energy = np.min(eigenvalues)
    excited_state_energies = np.sort(eigenvalues)[1:]
    return ground_state_energy, excited_state_energies




h_values = np.linspace(0, 2, 50)
ground_state_energies = []
excited_state_energies_list = [[] for _ in range(7)]

for h in h_values:
    H_sparse = ising_sparse(L, h, periodic=True)
    ground_energy, excited_energies = find_excited_energies(H_sparse)
    ground_state_energies.append(ground_energy)
    for i, energy in enumerate(excited_energies):
        excited_state_energies_list[i].append(energy)

ground_state_energies = np.array(ground_state_energies)
excited_state_energies = np.array(excited_state_energies_list)

plt.figure(figsize=(10, 8))
for i in range(7):
    energy_gaps = excited_state_energies[i] - ground_state_energies
    plt.plot(h_values, energy_gaps, label=f'Excited state {i+1}')

plt.axvline(x=1.0, color='r', linestyle='--', label='$h_c$')
plt.xlabel('Transverse field $h$')
plt.ylabel('Excitation energy $\Delta$')
plt.title('Excitation Energy vs Transverse Field for First 7 Excited States')
plt.legend()
plt.show()


In [ ]:
def fit_critical_exponent(h_values, energy_gaps, hc):
    fit_func = lambda h, nu: np.abs(h - hc) ** nu
    popt, pcov = curve_fit(fit_func, h_values, energy_gaps)
    return popt[0]
hc=1
energy_gaps_first_excited = excited_state_energies[0] - ground_state_energies

nu_estimate_first_excited = fit_critical_exponent(h_values, energy_gaps_first_excited, hc)

print(f"critical exponent nu for the first excited state: {nu_estimate_first_excited}")


Study of the fidelity

In [ ]:
def find_ground_state(H_sparse):
    eigenvalues, eigenvectors = sla.eigsh(H_sparse, k=1, which='SA')
    ground_state_energy = eigenvalues[0]
    ground_state_wavefunction = eigenvectors[:, 0]
    return ground_state_energy, ground_state_wavefunction
h_values = np.linspace(0.25, 2, 70)
fidelities = []
previous_ground_state = None

for h in h_values:
    H_sparse = ising_sparse(L, h, periodic=True)
    ground_energy, ground_state = find_ground_state(H_sparse)

    if previous_ground_state is not None:

        fidelity = np.abs(np.vdot(previous_ground_state, ground_state))
        fidelities.append(fidelity)

    previous_ground_state = ground_state


h_values_for_fidelity = h_values[1:]

plt.figure(figsize=(10, 8))
plt.plot(h_values_for_fidelity, fidelities, label='Fidelity')
plt.xlabel('Transverse field $h$')
plt.ylabel('Fidelity $|\\langle gs(h)|gs(h + \delta h)\\rangle|$')
plt.title('Fidelity as a function of Transverse Field')
plt.legend()
plt.show()



**4.5 Study of magnetic ordering**



In [ ]:
def expectation_value_sigma_z(psi, site1, site2, L):
    expectation_value = 0
    for a in range(len(psi)):

        site1 = (site1 - 1) % L
        site2 = (site2 - 1) % L

        spin1 = 1 if (a >> site1) & 1 else -1
        spin2 = 1 if (a >> site2) & 1 else -1

        contribution = spin1 * spin2 * (abs(psi[a]) ** 2)
        expectation_value += contribution

    return expectation_value




Correlation function as a function of h for 9 values of r

In [ ]:

from tqdm import tqdm

def compute_magnetization_squared(psi, L):
    M2 = 0
    for r in tqdm(range(L), desc='Magnetization Squared'):
        M2 += expectation_value_sigma_z(psi, 1, 1+r, L)
    return M2 / L
def compute_correlation_function(psi, L):
    Czz = np.zeros(L)
    for r in range(L):
        Czz[r] = expectation_value_sigma_z(psi, 1, 1+r, L)
    return Czz


L = 20
h_values = np.linspace(0, 2, 50)

M2_values = []
Czz_values = []

for h in tqdm(h_values, desc='Main Loop'):
    H_sparse = ising_sparse(L, h, periodic=True)
    _, psi_ground = find_ground_state(H_sparse)

    M2 = compute_magnetization_squared(psi_ground, L)
    M2_values.append(M2)

    Czz = compute_correlation_function(psi_ground, L)
    Czz_values.append(Czz)

for r in range(L//2):
    Czz_r = [Czz_values[i][r] for i in range(len(h_values))]
    plt.plot(h_values, Czz_r, label=f'r={r}')

plt.xlabel('Transverse field $h$')
plt.ylabel('Correlation $C_{zz}(r)$')
plt.title('Correlation Function $C_{zz}(r)$ for Different $r$')
plt.legend()
plt.show()



In [ ]:
def find_ground_state(H_sparse):
    eigenvalues, eigenvectors = eigsh(H_sparse, k=1, which='SA')
    ground_state_energy = eigenvalues[0]
    ground_state_wavefunction = eigenvectors[:, 0]
    return ground_state_energy, ground_state_wavefunction


Correlation function C^zz as a function of r for h=[0.3,1,1.7]

In [ ]:
h_values = [0.3, 1, 1.7]

Czz_values_for_h = {h: [] for h in h_values}
L_values=[10,15,20]
for L in L_values:
  max_r = L


  for h in tqdm(h_values, desc='Computing Czz for specific h values'):
      H_sparse = ising_sparse(L, h, periodic=True)
      _, psi_ground = find_ground_state(H_sparse)
      Czz_values_for_h[h] = compute_correlation_function(psi_ground, L)

  plt.figure(figsize=(10, 8))

  for h in h_values:
      plt.plot(range(max_r), Czz_values_for_h[h], label=f'h={h}')

  plt.xlabel('Site separation $r$')
  plt.ylabel('Correlation $C_{zz}(r)$')
  plt.title('Correlation Function $C_{zz}(r)$ for Different $h$')
  plt.legend()
  plt.show()

**4.7 Making use of the ising symmetry**

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
import scipy.sparse

def create_hamiltonian(L, h, periodic):

    size = 2 ** (L - 1)
    rows_even, cols_even, data_even = [], [], []
    rows_odd, cols_odd, data_odd = [], [], []


    for state in range(2 ** L):
        parity = (-1) ** bin(state).count("1")

        index = state >> 1

        diagonal_value = -h * (L - 2 * bin(state).count("1"))

        if parity == 1:
            rows_even.append(index)
            cols_even.append(index)
            data_even.append(diagonal_value)
        else:
            rows_odd.append(index)
            cols_odd.append(index)
            data_odd.append(diagonal_value)


        for j in range(L - 1):
            # flip j and j+1 bits
            flipped_state = state ^ (3 << j)
            new_parity = (-1) ** bin(flipped_state).count("1")
            new_index = flipped_state >> 1

            if new_parity == parity:
                if parity == 1:
                    rows_even.append(index)
                    cols_even.append(new_index)
                    data_even.append(-1)
                else:
                    rows_odd.append(index)
                    cols_odd.append(new_index)
                    data_odd.append(-1)

        if periodic and L > 1:
            flipped_state = state ^ (1 + (1 << (L - 1)))
            new_parity = (-1) ** bin(flipped_state).count("1")
            new_index = flipped_state >> 1

            if new_parity == parity:
                if parity == 1:
                    rows_even.append(index)
                    cols_even.append(new_index)
                    data_even.append(-1)
                else:
                    rows_odd.append(index)
                    cols_odd.append(new_index)
                    data_odd.append(-1)

    H_even = csr_matrix((data_even, (rows_even, cols_even)), shape=(size, size), dtype=np.float64)
    H_odd = csr_matrix((data_odd, (rows_odd, cols_odd)), shape=(size, size), dtype=np.float64)

    return H_even, H_odd






In [ ]:
import numpy as np
import scipy.sparse.linalg
from tqdm.notebook import tqdm


L_values = [8, 10, 12, 14, 16, 18]
h_values = np.linspace(-2,2,50)
ground_state_energies_even = {L: [] for L in L_values}
ground_state_energies_odd = {L: [] for L in L_values}


for L in L_values:
    for h in tqdm(h_values, desc=f'Computing for L={L}'):
        H_even, H_odd = create_hamiltonian(L, h, periodic=True)

        ground_state_even = scipy.sparse.linalg.eigsh(H_even, k=1, which='SA', return_eigenvectors=False)[0]
        ground_state_energies_even[L].append(ground_state_even)

        ground_state_odd = scipy.sparse.linalg.eigsh(H_odd, k=1, which='SA', return_eigenvectors=False)[0]
        ground_state_energies_odd[L].append(ground_state_odd)


ground_state_energies = {L: [] for L in L_values}
for L in L_values:
    for e_even, e_odd in zip(ground_state_energies_even[L], ground_state_energies_odd[L]):
        ground_state_energies[L].append(min(e_even, e_odd))


for L, energies in ground_state_energies.items():
    plt.plot(h_values, energies, label=f'L={L}')

plt.xlabel('h values')
plt.ylabel('Ground state energy of H')
plt.legend()
plt.show()




In [ ]:
def plot_energy_splitting(max_L, h, periodic):
    L_values = range(2, max_L + 1)
    splittings = []

    for L in L_values:
        H_even, H_odd = create_hamiltonian(L, h, periodic)
        ground_state_energy_even = scipy.sparse.linalg.eigsh(H_even, k=1, which='SA')[0][0]
        ground_state_energy_odd = scipy.sparse.linalg.eigsh(H_odd, k=1, which='SA')[0][0]
        splitting = abs(ground_state_energy_even - ground_state_energy_odd)
        splittings.append(splitting)

    plt.figure(figsize=(10, 5))
    plt.plot(L_values, splittings, marker='')
    plt.title('Energy Splitting vs System Size L')
    plt.xlabel('System Size L')
    plt.ylabel('Energy Splitting')
    plt.show()

In [ ]:
plot_energy_splitting(max_L=18, h=1.0, periodic=True)